구현내용
+ [x] 1.일별 Trend(지난주)
+ [x] 2.상세내용
  + [x] A.오피킹(오출고) 
  + [x] B.파손
  + [x] C.재출고 


In [44]:
###############
# 1. import
#####

import pandas as pd
from datetime import datetime, timedelta
import re
import time 
from bs4 import BeautifulSoup

# local only
import os
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)

pd.options.display.float_format = '{:0,.0f}'.format


In [3]:
# DB연결
conn = pymysql.connect(host = '**.***.*.*', user ='__USER_ID__', password = "__PASSWORD__", database='__DATABASE__')

# cursor 설정
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [4]:
sql= '''select code as 클레임유형, CODE_DC 
        from JT_CODE jc 
        where CODE_GRP_ID = 'CS_GUBUN'
    '''

cursor.execute(sql)
code = cursor.fetchall()
code = pd.DataFrame(code)

In [5]:
# code

In [10]:
###############
# 2.상세내용 : 전주
#####

#자료 불러오기 : 오피킹 자료 호출
sql= '''select (SELECT CODE_NM FROM JT_CODE B WHERE A.CS_GUBUN = B.CODE AND B.CODE_GRP_ID = 'CS_GUBUN') as 클레임유형
        , (case when DEALER_ID = ' ' then '기타'
                else (SELECT CODE_NM FROM JT_CODE B WHERE A.DEALER_ID = B.CODE AND B.CODE_GRP_ID = 'DEALER_ID') end )  체널
        , (case when RECEIPT = '01' then '유선' else '온라인' end) as 접수경로
        , concat( DATE_FORMAT(LAST_UPDUSR_PNTTM, '%m/%d') , ', ' 
        , (case when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 1 then '(월)' 
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 2 then '(화)'  
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 3 then '(수)'  
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 4 then '(목)'  
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 5 then '(금)' end) ) as 일자
        , concat(weekofyear(LAST_UPDUSR_PNTTM), ' 주차') as 주차
        , 1 as CNT
        FROM MAEIL_CS A
        where A.CS_STATUS in ('03','04','05')
        and DATE_FORMAT(LAST_UPDUSR_PNTTM, '%Y%m%d') between DATE_ADD(curdate(),INTERVAL -15 DAY) and DATE_ADD(curdate(),INTERVAL -9 DAY)
        '''

cursor.execute(sql)
result0 = cursor.fetchall()
result0 = pd.DataFrame(result0)

In [11]:
###############
# 2.상세내용 : 금주
#####

#자료 불러오기 : 오피킹 자료 호출
sql= '''select (SELECT CODE_NM FROM JT_CODE B WHERE A.CS_GUBUN = B.CODE AND B.CODE_GRP_ID = 'CS_GUBUN') as 클레임유형
        , (case when DEALER_ID = ' ' then '기타'
                else (SELECT CODE_NM FROM JT_CODE B WHERE A.DEALER_ID = B.CODE AND B.CODE_GRP_ID = 'DEALER_ID') end )  체널
        , (case when RECEIPT = '01' then '유선' else '온라인' end) as 접수경로
        , concat( DATE_FORMAT(LAST_UPDUSR_PNTTM, '%m/%d') , ', ' 
        , (case when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 1 then '(월)' 
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 2 then '(화)'  
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 3 then '(수)'  
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 4 then '(목)'  
                when DATE_FORMAT(LAST_UPDUSR_PNTTM, '%w') = 5 then '(금)' end) ) as 일자
        , concat(weekofyear(LAST_UPDUSR_PNTTM), ' 주차') as 주차
        , 1 as CNT
        FROM MAEIL_CS A
        where A.CS_STATUS in ('03','04','05')
        and DATE_FORMAT(LAST_UPDUSR_PNTTM, '%Y%m%d') between DATE_ADD(curdate(),INTERVAL -8 DAY) and DATE_ADD(curdate(),INTERVAL -2 DAY)
        '''

cursor.execute(sql)
result1 = cursor.fetchall()
result1 = pd.DataFrame(result1)

In [12]:
result1.head()

,클레임유형,체널,접수경로,일자,주차,CNT
0,906] 기타문의,26 셀렉스 몰,유선,"04/07, (수)",14 주차,1
1,204] 배송지연(주문건수↑),기타,유선,"04/06, (화)",14 주차,1
2,801] 배송단순문의,17 카카오 메이커스,유선,"04/06, (화)",14 주차,1
3,801] 배송단순문의,10 위메프,유선,"04/07, (수)",14 주차,1
4,401] 택배사 파손,3 네이버,유선,"04/08, (목)",14 주차,1


In [57]:
# 지난주 실적
df1_before = pd.pivot_table(result0, values = 'CNT', index=['접수경로'], columns=['일자' ], aggfunc=np.sum , margins = True).fillna(0)
count = len(df1_before.columns)-1
df1_before['일평균(A)'] = df1_before['All']/count

In [58]:
df1_before

일자,"03/29, (월)","03/30, (화)","03/31, (수)","04/01, (목)","04/02, (금)",All,일평균(A)
접수경로,,,,,,,
온라인,204,161,226,198,165,954,191
유선,43,63,75,41,62,284,57
All,247,224,301,239,227,1238,248


In [67]:
def draw_color_at_maxmum(x,color):
    color = f'background-color:{color}'
    is_max = x == x.max()
    return [color if b else '' for b in is_max]

In [65]:
# 이번주 실적
df1_this = pd.pivot_table(result1, values = 'CNT', index=['접수경로'], columns=['일자' ], aggfunc=np.sum , margins = True).fillna(0)
df1_this = 
count = len(df1_this.columns)-1
df1_this['일평균(B)'] = df1_this['All']/count

# 증감율 자료 추가
df1_this['증감율(A-B)'] = (df1_this['일평균(B)']- df1_before['일평균(A)'])/df1_before['일평균(A)']*100

In [71]:
df1_this['증감율(A-B)'].style.format("{:.2%}")

In [ ]:
df1_this.style.apply(draw_color_at_maxmum,color='#ff9090',axis=1)

In [73]:
pd.options.display.float_format = '{:0,.0f}'.format

df1_this = pd.pivot_table(result1, values = 'CNT', index=['접수경로'], columns=['일자' ], aggfunc=np.sum ).fillna(0)
df1_this = df1_this.style.apply(draw_color_at_maxmum,color='#ff9090',axis=1)

count = len(df1_this.columns)-1

In [84]:
df1_this.columns[0]

'04/05, (월)'

In [57]:
df2 = pd.pivot_table(result1, values = 'CNT', index=['체널'], columns=['일자' ], aggfunc=np.sum , margins = True).fillna(0)
df2

일자,"04/05, (월)","04/06, (화)","04/07, (수)","04/08, (목)","04/09, (금)",All
체널,,,,,,
1 쿠팡,2.0,4.0,0.0,2.0,2.0,10
10 위메프,7.0,6.0,14.0,10.0,9.0,46
13 다이렉트몰,0.0,2.0,8.0,11.0,2.0,23
15 맘큐,2.0,2.0,0.0,0.0,0.0,4
16 CJ 오쇼핑,17.0,11.0,38.0,11.0,7.0,84
17 카카오 메이커스,4.0,6.0,0.0,2.0,0.0,12
2 11번가,7.0,14.0,8.0,67.0,42.0,138
20 하이셀럽,0.0,0.0,0.0,2.0,0.0,2
21 카카오 선물하기,1.0,3.0,3.0,4.0,2.0,13


In [58]:
df3 = pd.pivot_table(result1, values = 'CNT', index=['클레임유형'], columns=['일자' ], aggfunc=np.sum , margins = True).fillna(0)
df3

일자,"04/05, (월)","04/06, (화)","04/07, (수)","04/08, (목)","04/09, (금)",All
클레임유형,,,,,,
101] 오매칭,0.0,0.0,0.0,1.0,0.0,1
104] 배송지연(재고부족),0.0,0.0,0.0,1.0,0.0,1
105] 배송지 기재 오류,0.0,1.0,0.0,0.0,0.0,1
106] 구매취소 후 출고,1.0,1.0,0.0,2.0,1.0,5
107] 주문수집누락,0.0,0.0,0.0,1.0,0.0,1
201] 오피킹,1.0,0.0,0.0,2.0,1.0,4
202] 배송누락(출고누락 등),0.0,2.0,3.0,2.0,4.0,11
204] 배송지연(주문건수↑),1.0,1.0,1.0,0.0,0.0,3
205] 운송장분실,0.0,0.0,1.0,0.0,1.0,2


In [5]:
# 데이터 보정

In [7]:
# result1

In [8]:
###############
# 2.상세내용 : 파손
#####

#자료 불러오기 : 오피킹 자료 호출
sql= '''SELECT T1.SEQ
        , T1.USER_NM as 처리담당자
        , DATE_FORMAT(T1.INS_DATE2, '%Y-%m-%d') AS 접수일자
        , (SELECT CODE_NM FROM JT_CODE WHERE CODE_WORK_GRP_ID = 'MAEIL' AND CODE_GRP_ID = 'DEALER_ID' AND CODE = T1.DEALERID) AS 판매체널
--         , (case when (case when CHAR_LENGTH(T1.ORDER_ID)=0 then null else T1.ORDER_ID end) is null then 
--           (case when CHAR_LENGTH(trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -2), '/', ''),'[가-힣 . ]',''))) between 7 and 20 
--                 then trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -2), '/', ''),'[가-힣 . ]',''))
--                 when CHAR_LENGTH(trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -3), '/', ''),'[가-힣]',''))) between 7 and 20  
--                 then trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -3), '/', ''),'[가-힣]',''))
--                 else NULL end ) 
--             else (case when CHAR_LENGTH(T1.ORDER_ID)=0 then null else T1.ORDER_ID end) end ) as 주문번호
        , REGEXP_SUBSTR(contents , '[0-9]{3}') as 클레임유형
        , '' as 클레임유형명
--         , trim(SUBSTRING_INDEX(SUBSTRING_INDEX(contents, '/', 3),'/', -1)) as 교환제품
        , REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '[ > ][0-9 A-Z]+ /') , '[0-9A-Z]+') as 주문번호_수정
        , (case when REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '(/ [0-9]+ [ > ] )|(/ [0-9]+[>])') , '[0-9A-Z]+') is null 
                then REGEXP_SUBSTR(contents , '(([G_].+))[>] ')
                else REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '(/ [0-9]+ [ > ] )|(/ [0-9]+[>])') , '[0-9A-Z]+') end )as 출고상품
        , (case when REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '([>]([0-9]+))|([ > ] ([0-9]+))'), '[0-9A-Z]+') is null 
                then REGEXP_SUBSTR(contents , '([>] ([G_].+)/)') 
                else REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '([>]([0-9]+))|([ > ] ([0-9]+))'), '[0-9A-Z]+') end )as 재출고상품
--         , REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '([>]([0-9]+))|([ > ] ([0-9]+))'), '[0-9A-Z]+') as 재출고상품
--         , REGEXP_SUBSTR(contents , '([>] ([G_].+)/)') as 재출고상품2
        , T1.MEMO_SEQ as 처리순차
        , T1.CONTENTS as "처리내역(원문)"
        , (SELECT CODE_NM FROM JT_CODE WHERE CODE_WORK_GRP_ID = 'MAEIL' AND CODE_GRP_ID = 'CS_STATUS' AND CODE = T1.CS_STATUS_NM) AS 처리상태
        FROM  ( SELECT A.SEQ
                      ,A.GUBUN
                      ,B.ORDER_ID
                      ,(CASE
                            WHEN C.CLAIM_INS_DATE IS NOT NULL THEN C.CLAIM_INS_DATE
                            WHEN B.INS_DM         IS NOT NULL THEN B.INS_DM
                            END) AS INS_DATE2
                       ,(CASE
                            WHEN B.DEALER_ID IS NOT NULL AND A.GUBUN = 'Q' THEN B.DEALER_ID
                            WHEN C.DEALER_ID IS NOT NULL AND A.GUBUN = 'C' THEN C.DEALER_ID
                            END) AS DEALERID
                       , (CASE
                            WHEN B.CS_GUBUN     IS NOT NULL THEN B.CS_GUBUN
                            WHEN C.GUBUN_DETAIL IS NOT NULL THEN C.GUBUN_DETAIL
                            END) AS CS_GUBUN2
                       , (CASE
                            WHEN B.RECEIPT IS NOT NULL AND A.GUBUN = 'Q' THEN B.RECEIPT
                            WHEN C.RECEIPT IS NOT NULL AND A.GUBUN = 'C' THEN C.RECEIPT
                            END) AS RECEIPT2
                       , (CASE
                            WHEN B.CNTS          IS NOT NULL AND A.GUBUN = 'Q'  THEN B.CNTS 
                            WHEN C.CLAME_CONTENT IS NOT NULL AND A.GUBUN = 'C' THEN C.CLAME_CONTENT
                            END) AS CNTS
                       ,(CASE
                            WHEN C.STATUS IS NOT NULL AND A.GUBUN = 'C'  THEN C.STATUS
                            WHEN B.CS_STATUS IS NOT NULL AND A.GUBUN = 'Q'  THEN B.CS_STATUS
                            END) AS CS_STATUS_NM
                       , A.MEMO_SEQ 
                       , A.CONTENTS 
                       , IFNULL((SELECT USER_NM FROM JT_USER WHERE INT_SVC_NO = A.FRST_REGISTER_NO), A.FRST_REGISTER_NO) AS USER_NM
                       , DATE_FORMAT(A.LAST_UPDUSR_PNTTM  ,'%Y%m%d%H%i%s') AS LAST_UPDUSR_PNTTM2
                       , C.GUBUN_DETAIL
                       , A.REGISTER_ID
                    FROM MAEIL_DETAIL A 
                    LEFT OUTER JOIN MAEIL_CS B 
                    ON A.SEQ = B.SEQ 
                    AND  A.REGISTER_ID = B.REGISTER_ID
                    LEFT OUTER JOIN MAEIL_CLAIM C 
                    ON A.SEQ = C.SEQ 
                    AND  A.REGISTER_ID = C.REGISTER_ID ) T1
                where CONTENTS like '%401] 택배사 파손 ]%'
                and DATE_FORMAT(substr(INS_DATE2,1,8), '%Y%m%d') between DATE_SUB(CURDATE(), INTERVAL 10 DAY) and DATE_SUB(CURDATE(), INTERVAL 3 DAY)
                ORDER BY T1.SEQ, T1.MEMO_SEQ asc
        '''

cursor.execute(sql)
result2 = cursor.fetchall()
result2 = pd.DataFrame(result2)

In [9]:
# 데이터 보정

# 중복제거 : 주문번호 기준
result2 = result2.drop_duplicates(['주문번호_수정'])

# 클레임유형 표기
result2['클레임유형'] = '401'

result2 = pd.merge(result2, code, on ='클레임유형', how = 'left')
result2['클레임유형명'] = result2['CODE_DC']
result2 = result2[result2.columns[:-1]]
result2['구분']  ='2.파손'

In [10]:
# result2

In [11]:
###############
# 2.상세내용 : 재출고 
#####

#자료 불러오기 : 오피킹 자료 호출
sql= '''SELECT T1.SEQ
        , T1.USER_NM as 처리담당자
        , DATE_FORMAT(T1.INS_DATE2, '%Y-%m-%d') AS 접수일자
        , (SELECT CODE_NM FROM JT_CODE WHERE CODE_WORK_GRP_ID = 'MAEIL' AND CODE_GRP_ID = 'DEALER_ID' AND CODE = T1.DEALERID) AS 판매체널
--         , (case when (case when CHAR_LENGTH(T1.ORDER_ID)=0 then null else T1.ORDER_ID end) is null then 
--           (case when CHAR_LENGTH(trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -2), '/', ''),'[가-힣 . ]',''))) between 7 and 20 
--                 then trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -2), '/', ''),'[가-힣 . ]',''))
--                 when CHAR_LENGTH(trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -3), '/', ''),'[가-힣]',''))) between 7 and 20  
--                 then trim(REGEXP_REPLACE(REPLACE(SUBSTRING_INDEX(contents, '/', -3), '/', ''),'[가-힣]',''))
--                 else NULL end ) 
--             else (case when CHAR_LENGTH(T1.ORDER_ID)=0 then null else T1.ORDER_ID end) end ) as 주문번호
        , REGEXP_SUBSTR(contents , '[0-9]{3}') as 클레임유형
        , '' as 클레임유형명
--         , trim(SUBSTRING_INDEX(SUBSTRING_INDEX(contents, '/', 3),'/', -1)) as 교환제품
        , REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '[ > ][0-9 A-Z]+ /') , '[0-9A-Z]+') as 주문번호_수정
        , (case when REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '(/ [0-9]+ [ > ] )|(/ [0-9]+[>])') , '[0-9A-Z]+') is null 
                then REGEXP_SUBSTR(contents , '(([G_].+))[>] ')
                else REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '(/ [0-9]+ [ > ] )|(/ [0-9]+[>])') , '[0-9A-Z]+') end )as 출고상품
        , (case when REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '([>]([0-9]+))|([ > ] ([0-9]+))'), '[0-9A-Z]+') is null 
                then REGEXP_SUBSTR(contents , '([>] ([G_].+)/)') 
                else REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '([>]([0-9]+))|([ > ] ([0-9]+))'), '[0-9A-Z]+') end )as 재출고상품
--         , REGEXP_SUBSTR(REGEXP_SUBSTR(contents , '([>]([0-9]+))|([ > ] ([0-9]+))'), '[0-9A-Z]+') as 재출고상품
--         , REGEXP_SUBSTR(contents , '([>] ([G_].+)/)') as 재출고상품2
        , T1.MEMO_SEQ as 처리순차
        , T1.CONTENTS as "처리내역(원문)"
        , (SELECT CODE_NM FROM JT_CODE WHERE CODE_WORK_GRP_ID = 'MAEIL' AND CODE_GRP_ID = 'CS_STATUS' AND CODE = T1.CS_STATUS_NM) AS 처리상태
        FROM  ( SELECT A.SEQ
                      ,A.GUBUN
                      ,B.ORDER_ID
                      ,(CASE
                            WHEN C.CLAIM_INS_DATE IS NOT NULL THEN C.CLAIM_INS_DATE
                            WHEN B.INS_DM         IS NOT NULL THEN B.INS_DM
                            END) AS INS_DATE2
                       ,(CASE
                            WHEN B.DEALER_ID IS NOT NULL AND A.GUBUN = 'Q' THEN B.DEALER_ID
                            WHEN C.DEALER_ID IS NOT NULL AND A.GUBUN = 'C' THEN C.DEALER_ID
                            END) AS DEALERID
                       , (CASE
                            WHEN B.CS_GUBUN     IS NOT NULL THEN B.CS_GUBUN
                            WHEN C.GUBUN_DETAIL IS NOT NULL THEN C.GUBUN_DETAIL
                            END) AS CS_GUBUN2
                       , (CASE
                            WHEN B.RECEIPT IS NOT NULL AND A.GUBUN = 'Q' THEN B.RECEIPT
                            WHEN C.RECEIPT IS NOT NULL AND A.GUBUN = 'C' THEN C.RECEIPT
                            END) AS RECEIPT2
                       , (CASE
                            WHEN B.CNTS          IS NOT NULL AND A.GUBUN = 'Q'  THEN B.CNTS 
                            WHEN C.CLAME_CONTENT IS NOT NULL AND A.GUBUN = 'C' THEN C.CLAME_CONTENT
                            END) AS CNTS
                       ,(CASE
                            WHEN C.STATUS IS NOT NULL AND A.GUBUN = 'C'  THEN C.STATUS
                            WHEN B.CS_STATUS IS NOT NULL AND A.GUBUN = 'Q'  THEN B.CS_STATUS
                            END) AS CS_STATUS_NM
                       , A.MEMO_SEQ 
                       , A.CONTENTS 
                       , IFNULL((SELECT USER_NM FROM JT_USER WHERE INT_SVC_NO = A.FRST_REGISTER_NO), A.FRST_REGISTER_NO) AS USER_NM
                       , DATE_FORMAT(A.LAST_UPDUSR_PNTTM  ,'%Y%m%d%H%i%s') AS LAST_UPDUSR_PNTTM2
                       , C.GUBUN_DETAIL
                       , A.REGISTER_ID
                    FROM MAEIL_DETAIL A 
                    LEFT OUTER JOIN MAEIL_CS B 
                    ON A.SEQ = B.SEQ 
                    AND  A.REGISTER_ID = B.REGISTER_ID
                    LEFT OUTER JOIN MAEIL_CLAIM C 
                    ON A.SEQ = C.SEQ 
                    AND  A.REGISTER_ID = C.REGISTER_ID ) T1
                where CONTENTS like '%재출고%'
                and CONTENTS not like '%[ 401] 택배사 파손 ]%'
                and DATE_FORMAT(substr(INS_DATE2,1,8), '%Y%m%d') between DATE_SUB(CURDATE(), INTERVAL 10 DAY) and DATE_SUB(CURDATE(), INTERVAL 3 DAY)
                ORDER BY T1.SEQ, T1.MEMO_SEQ asc
        '''

cursor.execute(sql)
result3 = cursor.fetchall()
result3 = pd.DataFrame(result3)

In [12]:
# 중복제거 : 주문번호 기준
result3 = result3.drop_duplicates(['주문번호_수정'])

# 클레임유형 표기
result3 = pd.merge(result3, code, on ='클레임유형', how = 'left')
result3['클레임유형명'] = result3['CODE_DC']
result3 = result3[result3.columns[:-1]]
result3['구분']  ='3.재출고'

In [13]:
# result3

In [14]:
len(result3)

12

In [15]:
result = pd.concat([result1, result2, result3], ignore_index=True )

In [16]:
df1 = pd.DataFrame(result.groupby('구분')['클레임유형명'].count())
df1

,클레임유형명
구분,
1.오피킹,3
2.파손,30
3.재출고,12


In [110]:
df2 = pd.DataFrame(result.groupby(['구분', '클레임유형명'])['클레임유형명'].count())
df2

클레임유형명
구분    클레임유형명              
1.오피킹 오피킹                3
2.파손  택배사 파손            30
3.재출고 \t 제품불량(클레임)       1
      CS                 1
      배송누락(출고누락 등)       5
      운송장분실              1
      택배사 분실             4

In [19]:
df3 = pd.crosstab(result.구분, result.관리일자)

In [20]:
df3

관리일자,2021-03-28,2021-03-29,2021-03-30,2021-03-31,2021-04-01,2021-04-02,2021-04-03
구분,,,,,,,
1.오피킹,1,0,1,1,0,0,0
2.파손,0,5,11,1,5,7,1
3.재출고,0,4,1,5,1,1,0


In [34]:
df1 = pd.pivot_table(result, values = '처리순차', index='구분', columns='관리일자', aggfunc='count' , margins = True).fillna(0)

In [38]:
result.관리일자	.min(), result.관리일자	.max()

('2021-03-28', '2021-04-03')

In [ ]:
1. 